In [1]:
import torch

In [2]:
a = torch.randint(0, 9, size=(40000, 35000))
a = a.to('cuda:1')

In [10]:
torch.cuda.memory_allocated(0), torch.cuda.get_device_properties(0).total_memory

(0, 25388515328)

In [15]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(1)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')
print(info.free / info.total)

total    : 25769803776
free     : 8190820352
used     : 17578983424
0.3178456624348958


# Load other models

In [1]:
import warnings

from diffusers import DiffusionPipeline
from diffusers import StableDiffusionPipeline
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from diffusers.utils import logging
from transformers import CLIPImageProcessor

import torch
import safetensors


logging.set_verbosity_error()
warnings.filterwarnings("ignore")


/local/disk5/chrenx/conda-env/envs/humanai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt = "Sherlock Holmes"
negative_prompt = ""

safetensor_path = 'models/pencil_sketch.safetensors'

device_id = 2

before = torch.cuda.get_device_properties(device_id).total_memory - torch.cuda.memory_allocated(7)
before /= (1024 ** 2)
#*********************

# pipeline = DiffusionPipeline.from_pretrained(safetensor_path, torch_dtype=torch.float16, use_safetensors=True)


pipeline = StableDiffusionPipeline.from_single_file(safetensor_path,
                                                    extract_ema=True,
                                                    safety_checker=StableDiffusionSafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker"),
                                                    feature_extractor=CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32"),
                                                    image_size=512)

pipeline.to(f"cuda:{device_id}")

after = torch.cuda.get_device_properties(device_id).total_memory - torch.cuda.memory_allocated(7)
after /= (1024 ** 2)

print(f"usage: {(before-after):.2f} MB")

usage: 0.00 MB


# Example to use safetensor with stable diffusion model

In [4]:
import warnings

from diffusers import StableDiffusionPipeline
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from diffusers.utils import logging
from transformers import CLIPImageProcessor
from PIL import Image 

import torch

logging.set_verbosity_error()
warnings.filterwarnings("ignore")

prompt = "asdqwe"
negative_prompt = ""

safetensor_path = "models/background_beautiful_outdoor.safetensors"

device_id = 1

before = torch.cuda.get_device_properties(device_id).total_memory - torch.cuda.memory_allocated(device_id)
before /= (1024 ** 2)
#*********************


pipeline = StableDiffusionPipeline.from_single_file(safetensor_path,
                                                    extract_ema=True,
                                                    safety_checker=StableDiffusionSafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker"),
                                                    feature_extractor=CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32"),
                                                    image_size=512)
device = f"cuda:{device_id}" if torch.cuda.is_available() else "cpu"
pipeline.to(device)

print(pipeline.device)

image = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt, 
    num_inference_steps=25, # sampling steps
    guidance_scale=7, # 7, # cfg
    generator=torch.manual_seed(246372),
    num_images_per_prompt=2,
    width= 384, # 360,
    height= 512, # 480
)

img0 = image.images[0].copy()
img1 = image.images[1].copy()

after = torch.cuda.get_device_properties(device_id).total_memory - torch.cuda.memory_allocated(device_id)
after /= (1024 ** 2)

torch.cuda.empty_cache()

print(f"usage: {(before-after):.2f} MB")

cuda:1


100%|██████████| 25/25 [00:05<00:00,  4.38it/s]


usage: 0.25 MB


In [5]:
len(image['nsfw_content_detected'])

4

In [5]:
for i in range(len(image.images)):
    img = image.images[i]
    img.save(f"output{i}.png")
    if image['nsfw_content_detected'][0]:
        print('yes')
    else:
        print('no')
# img = image.images[0]
# img.save("output.png")
# if image['nsfw_content_detected'][0]:
#     print('yes')
# else:
#     print('no')
# img.size, type(img)

no
no


# Debug

In [ ]:
a = [1, 4, 5]
b = 3
a + b

7500000000.0

In [10]:
import gradio as gr


def show_text(model_name):
    return gr.update()


demo = gr.Blocks()

with demo:
    gr.Markdown(
        """
    # Show text!
    Start typing below to see the output.
    """
    )
    model_name = gr.Dropdown(label="Select Your Imaging Style", 
                                         choices=[2, 4, 5, 3], value=None)

    model_name.change(fn=show_text, inputs=model_name, outputs=model_name, show_progress = True)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://cf03891c9a0af8d165.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
